In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 23 15:59:07 2017
@author: tomhope
"""

from __future__ import print_function
import os
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import mnist
import numpy as np

In [ ]:
## generating the training batches
import numpy as np
import scipy.io
from array import array
import tensorflow as tf
read_input = scipy.io.loadmat('data_format_2/train_32x32.mat')

class data_set(object):
    pass
result = data_set()

data_set.images = read_input['X'].reshape(73257,32,32,3)
data_set.labels = read_input['y']
data_set.number_of_examples = read_input['y'].shape[0]

In [ ]:

save_dir = "./test"

filename = os.path.join(save_dir,'train.tfrecords')
writer = tf.python_io.TFRecordWriter(filename)
for index in range(data_set.images.shape[0]):
    images = data_set.images[index].tostring()
    example = tf.train.Example(features=tf.train.Features(
        feature={
            'height': tf.train.Feature(
                int64_list=tf.train.Int64List(
                    value=[data_set.images.shape[1]])),
            'width': tf.train.Feature(
                int64_list=tf.train.Int64List(
                    value=[data_set.images.shape[2]])),
            'depth': tf.train.Feature(
                int64_list=tf.train.Int64List(
                    value=[data_set.images.shape[3]])),
            'label': tf.train.Feature(
                int64_list=tf.train.Int64List(
                    value=[int(data_set.labels[index])])),
            'image_raw': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[images]))}))
    writer.write(example.SerializeToString())
writer.close()

In [ ]:
filename = os.path.join(save_dir, 'train.tfrecords')
record_iterator = tf.python_io.tf_record_iterator(filename)
seralized_img_example = next(record_iterator)

example = tf.train.Example()
example.ParseFromString(seralized_img_example)
image = example.features.feature['image_raw'].bytes_list.value
label = example.features.feature['label'].int64_list.value[0]
width = example.features.feature['width'].int64_list.value[0]
height = example.features.feature['height'].int64_list.value[0]

img_flat = np.frombuffer(image[0], dtype=np.uint8)
img_reshaped = img_flat.reshape((height, width, -1))
print(img_flat)
print(label)

In [ ]:

sess= tf.InteractiveSession()

In [ ]:

# READ
NUM_EPOCHS = 10

filename = os.path.join("./test", "train.tfrecords")

filename_queue = tf.train.string_input_producer(
    [filename], num_epochs=NUM_EPOCHS)


reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized_example,
    features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64),
    })

image = tf.decode_raw(features['image_raw'], tf.uint8)
image.set_shape([32*32*3])


image = tf.cast(image, tf.float32) * (1. / 255) - 0.5

label = tf.cast(features['label'], tf.int32)


# Shuffle the examples + batch
images_batch, labels_batch = tf.train.shuffle_batch(
    [image, label], batch_size=128,
    capacity=2000,
    min_after_dequeue=1000)


W = tf.get_variable("W", [32*32*3, 10])
y_pred = tf.matmul(images_batch, W)
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_pred, labels=labels_batch)

loss_mean = tf.reduce_mean(loss)

train_op = tf.train.AdamOptimizer().minimize(loss)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
init = tf.local_variables_initializer()
sess.run(init)




In [ ]:

# coordinator
coord = tf.train.Coordinator()
try:
    step = 0
    while not coord.should_stop():
        step += 1
        sess.run([train_op])
        if step % 500 == 0:
            loss_mean_val = sess.run([loss_mean])
            print(step)
            print(loss_mean_val)
except tf.errors.OutOfRangeError:
    print('Done training for %d epochs, %d steps.' % (NUM_EPOCHS, step))
finally:
    # When done, ask the threads to stop.
    coord.request_stop()
    
# example -- get image,label
img1, lbl1 = sess.run([image, label])

# example - get random batch
labels, images = sess.run([labels_batch, images_batch])

In [ ]:
sess.run(train_op)

In [ ]:
label